Comience esta actividad observando cómo los árboles de decisión tienden a sobreajustar los datos si no se controlan. El ajuste excesivo de los datos significa que intentan dar cuenta de los puntos de datos atípicos a costa de la precisión de la predicción de la tendencia general.

También veremos la validación cruzada k -fold. Este es un método de validación más sólido que el método de retención que usamos anteriormente.

En la validación cruzada k -fold, podemos probar cada ejemplo una vez. Esto se hace dividiendo el conjunto de datos en k subconjuntos y entrenando/probando el modelo k veces usando diferentes combinaciones de los subconjuntos.

Finalmente, observamos qué tan preciso es nuestro modelo en los QSO en comparación con otras galaxias. Como se mencionó en las conferencias, los QSO son galaxias que tienen un Núcleo Galáctico Activo (AGN). El AGN hace que la galaxia sea más brillante y, como tal, son detectables con los instrumentos SDSS hasta desplazamientos al rojo mucho más altos.

Usaremos el mismo conjunto de datos que la primera actividad e incluso algunas de las funciones que escribimos en las preguntas anteriores.

Los árboles de decisión tienen muchas ventajas: son simples de implementar, fáciles de interpretar, los datos no requieren demasiada preparación y son computacionalmente eficientes.

Sin embargo, los árboles de decisión tienen algunas limitaciones, una de las más importantes es que tienden a ajustarse en exceso a los datos. Lo que esto significa es que si no se controlan, crearán un árbol demasiado complicado que intentará dar cuenta de los valores atípicos en los datos. Esto se produce a expensas de la precisión de la tendencia general.

Parte de la razón de este sobreajuste es que el algoritmo funciona tratando de optimizar la decisión localmente en cada nodo. Hay formas en las que esto se puede mitigar y en el próximo problema veremos cómo la restricción del número de filas de nodos de decisión (la profundidad del árbol) afecta la precisión de nuestras predicciones.

Para ver cómo se sobreajusta el árbol, nos gustaría examinar cómo funciona nuestro árbol de decisión para diferentes profundidades de árbol. Específicamente, nos gustaría ver cómo se desempeña en los datos de prueba en comparación con los datos que se usaron para entrenarlo.

Ingenuamente, esperaríamos que cuanto más profundo sea el árbol, mejor debería funcionar. Sin embargo, a medida que el modelo se sobreajusta, vemos una diferencia en su precisión en los datos de entrenamiento y los datos de prueba más generales.

Podemos controlar la profundidad del árbol de decisión aprendido, usando un argumento para DecisionTreeRegressor. Por ejemplo, para establecer la profundidad máxima en 5:

dtr = DecisionTreeRegressor(max_depth=5)

Completa la función accuracy_by_treedepth. La función debe devolver la diferencia mediana para los conjuntos de datos de prueba y entrenamiento para cada una de las profundidades del árbol en depths.

accuracy_by_treedepthdebe tomar los siguientes argumentos:

featuresy targets(como en problemas anteriores);
depths: una matriz de profundidades de árbol que se utilizará como max_depthregresor del árbol de decisión.
Su función debe devolver dos listas (o matrices) que contengan los median_diffvalores de las predicciones realizadas en los conjuntos de entrenamiento y prueba utilizando las profundidades de árbol máximas proporcionadas por depths.

Por ejemplo, si depthses , entonces su función debería devolver dos listas de longitud 3. Puede elegir el tamaño de la división entre sus datos de prueba y entrenamiento (en caso de duda, 50:50 está bien). [3, 5, 7]

Hemos incluido código para trazar las diferencias en función de las profundidades de los árboles. Debe tomarse un momento para familiarizarse con lo que hace cada línea. Si su código funciona bien, entonces su gráfico debería parecerse un poco a lo siguiente:

Gráfica de diferencia de medianas vs profundidad de árbol

Aquí usamos índices para acceder a los datos en nuestro y features, targetssin embargo, hubiera sido igualmente válido crear 4 matrices explícitas train_features, y , test_featurestrain_targetstest_targets

Construimos dos listas accuracies_traina accuracies_test las que podemos agregar los med_diffvalores para cada profundidad en el ciclo for.

El árbol de decisiones se instancia en cada iteración del ciclo con un nuevo max_depth que se toma del argumento en el ciclo for. También configuramos el . Esto se relaciona con los números pseudoaleatorios utilizados para generar el árbol y al establecerlo en 0 nos aseguramos de que todas las profundidades se prueben de manera consistente. Este no es un requisito de su solución, pero ciertamente se recomienda. random_seed=0

Luego entrenamos el árbol y obtenemos predicciones para las funciones de entrenamiento y prueba. Como se mencionó anteriormente, generalmente es malo usar los mismos datos de entrenamiento para evaluar la precisión del modelo, sin embargo, como se mencionó en el contexto de los problemas, los estamos usando para comparar.

Finalmente, usamos las predicciones para cada conjunto para calcular el med_diff(usando nuestra función escrita previamente) y lo agregamos a nuestro anterior. accuracies_*

Solución de muestra
solución.py
import numpy as np
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor
​
# paste your get_features_targets function here
def get_features_targets(data):
  features = np.zeros((data.shape[0], 4))
  features[:, 0] = data['u'] - data['g']
  features[:, 1] = data['g'] - data['r']
  features[:, 2] = data['r'] - data['i']
  features[:, 3] = data['i'] - data['z']
  targets = data['redshift']
  return features, targets
​
# paste your median_diff function here
def median_diff(predicted, actual):
  return np.median(np.abs(predicted - actual))
​
# Complete the following function
def accuracy_by_treedepth(features, targets, depths):
  # split the data into testing and training sets
  split = features.shape[0]//2
  train_features, test_features = features[:split], features[split:]
  train_targets, test_targets = targets[:split], targets[split:]
​
  # Initialise arrays or lists to store the accuracies for the below loop
  train_diffs = []
  test_diffs = []
​
  # Loop through depths
  for depth in depths:
    # initialize model with the maximum depth. 
    dtr = DecisionTreeRegressor(max_depth=depth)
​
    # train the model using the training set
    dtr.fit(train_features, train_targets)
​
    # Get the predictions for the training set and calculate their med_diff
    predictions = dtr.predict(train_features)
    train_diffs.append(median_diff(train_targets, predictions))
​
    # Get the predictions for the testing set and calculate their med_diff
    predictions = dtr.predict(test_features)
    test_diffs.append(median_diff(test_targets, predictions))
        
  # Return the accuracies for the training and testing sets
  return train_diffs, test_diffs    
​
if __name__ == "__main__":
  data = np.load('sdss_galaxy_colors.npy')
  features, targets = get_features_targets(data)
​
  # Generate several depths to test
  tree_depths = [i for i in range(1, 36, 2)]
​
  # Call the function
  train_med_diffs, test_med_diffs = accuracy_by_treedepth(features, targets, tree_depths)
  print("Depth with lowest median difference : {}".format(tree_depths[test_med_diffs.index(min(test_med_diffs))]))
    
  # Plot the results
  train_plot = plt.plot(tree_depths, train_med_diffs, label='Training set')
  test_plot = plt.plot(tree_depths, test_med_diffs, label='Validation set')
  plt.xlabel("Maximum Tree Depth")
  plt.ylabel("Median of Differences")
  plt.legend()
  plt.show()
Prueba esta solución

Sus resultados deben mostrar una gráfica similar a la siguiente:


Podemos ver que la precisión del árbol de decisión en el conjunto de entrenamiento mejora a medida que permitimos que el árbol crezca a mayor profundidad. De hecho, a una profundidad de 27, ¡nuestros errores se reducen a cero!

Por el contrario, la medida de precisión de las predicciones para el conjunto de prueba mejora inicialmente y luego empeora a mayores profundidades del árbol. A una profundidad de árbol de ~19, el árbol de decisión comienza a sobreajustarse a los datos. Esto significa que intenta tener en cuenta los valores atípicos en el conjunto de entrenamiento y pierde su precisión predictiva general.

El sobreajuste es un problema común con los árboles de decisión y se puede evitar ajustando parámetros como la profundidad del árbol o estableciendo un número mínimo de casos en cada nodo. Por ahora, estableceremos una profundidad de árbol máxima de 19 para evitar un ajuste excesivo en nuestro problema de corrimiento al rojo.

El método que usamos para validar nuestro modelo hasta ahora se conoce como validación de retención. La validación de espera divide los datos en dos, uno para probar y otro para entrenar. La validación de espera es la forma más básica de validación.

Si bien la validación de espera es mejor que ninguna validación, la precisión medida (es decir, nuestra mediana de diferencias) variará según cómo dividamos los datos en subconjuntos de prueba y entrenamiento. Lo med_diffque obtenemos de un conjunto de entrenamiento muestreado aleatoriamente variará al de un conjunto de entrenamiento aleatorio diferente del mismo tamaño.

Para estar más seguros de la precisión de nuestros modelos, debemos utilizar la validación cruzada  
k
  veces.  
k
  -fold funciona de manera similar a la retención, excepto que dividimos los datos en  
k
  subconjuntos. Entrenamos y probamos el modelo  
k
  veces, registrando la precisión cada vez. Cada vez usamos una combinación diferente de subconjuntos  
k
−
1
  para entrenar el modelo y el subconjunto final  
k
t
h
  para probar. Tomamos el promedio de las  
k
  mediciones de precisión como la precisión general del modelo.

KDoblar
La KFoldbiblioteca está diseñada para dividir los datos en subconjuntos de entrenamiento y prueba. Lo hace ofreciendo un objeto iterable que se puede inicializar con

kf = KFold(n_splits=k, shuffle=True)
El especifica el número de subconjuntos a usar. n_splits=k

Por defecto shufflese establece en falso. En general, es una buena práctica mezclar los datos para la validación cruzada, ya que a veces, durante la recopilación y el almacenamiento, los datos de un tipo similar se pueden almacenar de forma adyacente, lo que generaría un sesgo de aprendizaje al entrenar el árbol. Por ejemplo, si los datos se ordenaron por corrimiento al rojo, en la primera iteración el modelo podría entrenarse con corrimientos al rojo de 0 a 3 y probarse en galaxias con corrimientos al rojo ~4.

En los próximos problemas, usaremos la sklearnbiblioteca KFoldpara ayudarnos a dividir nuestros datos en nuestros y el subconjunto de prueba restante. En el primer problema usaremos la conveniencia de para ayudarnos a calcular la veces. En el segundo, extenderemos esto para proporcionar una plegada para cada galaxia en nuestro conjunto de datos.  
k
−
1
 KFolds 
k
  
k

Su tarea es completar la función cross_validate_model. La función toma 4 argumentos:

model, feaures, y targetscomo en problemas anteriores;
ken nuestro k -fold. Este es el número de subconjuntos para entrenar y probar.
Su función debe devolver una lista que contenga la k mediana de las diferencias para cada uno de los k pliegues usando median_diff.

Tenga en cuenta que hemos establecido cuándo inicializamos el árbol de decisión para evitar que el modelo se sobreajuste. max_depth=19

Uso de KFolds
Hemos creado el KFoldobjeto para brindarle un conjunto de índices de entrenamiento y prueba para cada una de las k ejecuciones. Vale la pena tomarse un momento para entender esto.

En concreto, el objeto se inicializa con

kf = KFold(n_splits=k, shuffle=True)
El pasa nuestro número deseado de subconjuntos/pliegues. Queremos barajar los datos (como se explicó anteriormente). A continuación, el iterador se utiliza con: n_splits=k

for train_indices, test_indices in kf.split(features):
El es un iterador que, para cada una de las k iteraciones, devuelve dos matrices de índices para usar con nuestras matrices de funciones y objetivos, es decir kf.split(features)features[train_indices],targets[train_indices]

Esto es bastante similar a nuestra implementación anterior para la validación en el último tutorial. Las principales diferencias son que estamos usando train_indicesy test_indicespara dividir nuestras funciones y matrices de destino.

Otra diferencia es que recopilamos nuestros med_diffvalores calculados en una lista que se devolverá al final de la función.

Solución de muestra
solución.py
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
​
# paste your get_features_targets function here
def get_features_targets(data):
  features = np.zeros((data.shape[0], 4))
  features[:, 0] = data['u'] - data['g']
  features[:, 1] = data['g'] - data['r']
  features[:, 2] = data['r'] - data['i']
  features[:, 3] = data['i'] - data['z']
  targets = data['redshift']
  return features, targets
​
# paste your median_diff function here
def median_diff(predicted, actual):
  return np.median(np.abs(predicted - actual))
​
# complete this function
def cross_validate_model(model, features, targets, k):
  kf = KFold(n_splits=k, shuffle=True)
​
  # initialise a list to collect median_diffs for each iteration of the loop below
  diffs = []
​
  for train_indices, test_indices in kf.split(features):
    train_features, test_features = features[train_indices], features[test_indices]
    train_targets, test_targets = targets[train_indices], targets[test_indices]
    
    # fit the model for the current set
    model.fit(train_features, train_targets)
    
    # predict using the model
    predictions = model.predict(test_features)
 
    # calculate the median_diff from predicted values and append to results array
    diffs.append(median_diff(predictions, test_targets))
 
  # return the list with your median difference values
  return diffs
​
​
if __name__ == "__main__":
  data = np.load('./sdss_galaxy_colors.npy')
  features, targets = get_features_targets(data)
​
  # initialize model with a maximum depth of 19
  dtr = DecisionTreeRegressor(max_depth=19)
​
  # call your cross validation function
  diffs = cross_validate_model(dtr, features, targets, 10)
​
  # Print the values
  print('Differences: {}'.format(', '.join(['{:.3f}'.format(val) for val in diffs])))
  print('Mean difference: {:.3f}'.format(np.mean(diffs)))
Prueba esta solución

La validación cruzada es una parte importante para garantizar que nuestro modelo devuelva valores que sean al menos parcialmente precisos. El problema con la validación retenida es que solo podemos obtener valores de predicción para los datos en nuestro conjunto de prueba.

Con la validación cruzada de k veces, cada galaxia se prueba al menos una vez y, debido a esto, podemos obtener un valor de predicción para cada galaxia. Lo haremos en la siguiente pregunta...

Completa la función cross_validate_predictions. Esto es muy similar a la pregunta anterior, excepto que en lugar de devolver las med_diffmedidas de precisión, nos gustaría devolver un valor predicho para cada una de las galaxias.

La función toma los mismos 4 argumentos que la pregunta anterior, es decir model, feaures, targetsy k.

Su función debe devolver una sola variable. La variable devuelta debe ser una matriz numpy 1-D de longitud , donde es el número de galaxias en nuestro conjunto de datos. Debes asegurarte de mantener el orden de las galaxias al dar tus predicciones, de modo que la primera predicción en tu matriz corresponda a la primera galaxia en las matrices y .  
metro
  
metro
 featurestargets

La validación cruzada de K-Fold es una parte importante de la evaluación de la precisión de cualquier modelo de aprendizaje automático. Cuando trazamos nuestros corrimientos al rojo predichos frente a los medidos, pudimos ver que para muchas de nuestras galaxias pudimos obtener una predicción razonablemente precisa del corrimiento al rojo. Sin embargo, también hay varios valores atípicos en los que nuestro modelo no da una buena predicción.

Gráfica de corrimiento al rojo pronosticado vs medido
Hemos aprendido el funcionamiento interno de la validación cruzada de  
k
 -Fold con la ayuda de la KFoldbiblioteca. Ahora que tiene una comprensión funcional de  
k
 -Fold, debe tener en cuenta que hay varios métodos y bibliotecas en los módulos que proporcionan versiones listas para usar de algunas de las rutinas que acabamos de escribir. sklearn.model_selection

La función realiza las mismas acciones que la función que escribiste en la pregunta anterior. Se puede llamar con cross_val_predictcross_validate_predictions

predictions = cross_val_predict(dtr, features, targets, cv=k)
Donde dtrestá nuestro objeto regresor del árbol de decisión, nos permite especificar el número de pliegues ( ) a usar y / son como los hemos usado hasta ahora. cv=kkfeaturestargets

Hay otra herramienta en la biblioteca que vale la pena mencionar, la función. Esto proporciona una puntuación de qué tan bien se desempeñó el modelo similar al que hemos estado usando hasta ahora. No entraremos en el uso aquí, pero debe especificar qué métrica se usa para calificar el modelo. sklearn.model_selectioncross_val_score med_diff

Puede que te sorprenda saber que nuestra muestra de galaxias consta de dos poblaciones diferentes: galaxias regulares y objetos cuasi-estelares (QSO). Los QSO son un tipo de galaxia que contiene un agujero negro supermasivo que se acumula activamente (e intensamente). Esto a menudo se denomina Núcleo Galáctico Activo (AGN).

Galaxia con un AGN
La luz emitida por el AGN es significativamente más brillante que el resto de la galaxia y podemos detectar estos QSO con desplazamientos al rojo mucho más altos. De hecho, la mayoría de las galaxias normales que hemos estado usando para crear nuestros modelos tienen corrimientos al rojo menores que z 0.4, mientras que los QSO tienen corrimientos al rojo hasta z 6. Debido a esta contribución del AGN, el flujo Es posible que las magnitudes medidas en diferentes longitudes de onda no sigan el perfil típico que asumimos al predecir los desplazamientos al rojo.  
≈
  
≈
 
En la siguiente pregunta, veremos si existe una diferencia en la precisión de los árboles de decisión entre los QSO y las galaxias regulares.

Escriba una función split_galaxies_qsosque divida nuestros datos que contienen tanto galaxias como QSO en dos matrices que contengan solo galaxias y QSO, respectivamente. Su función debe tomar un solo dataargumento.

La función debe devolver dos matrices NumPy, la primera que galaxiescontiene solo filas dataque son galaxias y la segunda qsosque contiene solo filas que son QSO.

La datamatriz contiene una columna donde los valores serán o . data['spec_class']b'GALAXY'b'QSO'

Gotcha: usar b'GALAXY'y no'GALAXY'
La clase espectral se almacena como una cadena de bytes (no cadenas Unicode), por lo que los literales deben tener un bal frente. Comparar contra 'GALAXY'no coincidirá con ninguna fila, mientras que b'GALAXY'sí.

Sugerencia: use enmascaramiento para seleccionar las filas
Podemos usar el enmascaramiento para seleccionar filas particulares:

 
import numpy as np
data = np.load('sdss_galaxy_colors.npy')
galaxies = data[data['spec_class'] == b'GALAXY']
El interior devuelve todos los índices que tienen un tipo espectral de galaxia. Estos índices se utilizan luego para seleccionar las filas con el exterior . data['spec_class'] == b'GALAXY'data[...]

Esta solución utiliza enmascaramiento.

qso_mask = data['spec_class'] == b'QSO'
Crea una matriz de valores booleanos Truesi y en caso contrario. data['spec_class'] == b'QSO'False

Hacemos lo mismo para las galaxias y luego creamos nuestras dos matrices de retorno usando las máscaras.

Solución de muestra
solución.py
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
​
# paste your get_features_targets function here
def get_features_targets(data):
  features = np.zeros((data.shape[0], 4))
  features[:, 0] = data['u'] - data['g']
  features[:, 1] = data['g'] - data['r']
  features[:, 2] = data['r'] - data['i']
  features[:, 3] = data['i'] - data['z']
  targets = data['redshift']
  return features, targets
​
# paste your median_diff function here
def median_diff(predicted, actual):
  return np.median(np.abs(predicted - actual))
​
# paste your cross_validate_model function here
def cross_validate_model(model, features, targets, k):
  kf = KFold(n_splits=k, shuffle=True)
​
  # initialise a list to collect median_diffs for each iteration of the loop below
  diffs = []
​
  for train_indices, test_indices in kf.split(features):
    train_features, test_features = features[train_indices], features[test_indices]
    train_targets, test_targets = targets[train_indices], targets[test_indices]
    
    # fit the model for the current set
    model.fit(train_features, train_targets)
    
    # predict using the model
    predictions = model.predict(test_features)
 
    # calculate the median_diff from predicted values and append to results array
    diffs.append(median_diff(predictions, test_targets))
 
  # return the list with your median difference values
  return diffs
​
# complete this function
def split_galaxies_qsos(data):
  # split the data into galaxies and qsos arrays
  galaxies = data[data['spec_class'] == b'GALAXY']
  qsos = data[data['spec_class'] == b'QSO']
​
  # return the seperated galaxies and qsos arrays
  return galaxies, qsos
​
def cross_validate_median_diff(data):
  features, targets = get_features_targets(data)
  dtr = DecisionTreeRegressor(max_depth=19)
  return np.mean(cross_validate_model(dtr, features, targets, 10))
​
if __name__ == "__main__":
  data = np.load('./sdss_galaxy_colors.npy')
​
  # split the data set into galaxies and QSOs
  galaxies, qsos= split_galaxies_qsos(data)
​
  # here we cross validate the model and get the cross-validated median difference
  # the cross_validated_med_diff function is in "written_functions"
  galaxy_med_diff = cross_validate_median_diff(galaxies)
  qso_med_diff = cross_validate_median_diff(qsos)
​
  # print the results
  print("Median difference for Galaxies: {:.3f}".format(galaxy_med_diff))
  print("Median difference for QSOs: {:.3f}".format(qso_med_diff))
Prueba esta solución

Así que nuestros QSO tienen una mediana residual mayor ( ) que las galaxias ( ). Hay un par de posibilidades de por qué este es el caso.  
≈
0.074
  
≈
0.016
 
Hay muchos menos QSO (8525) que galaxias (41,475).
Las galaxias no son tan brillantes como los QSO, por lo que se vuelven demasiado débiles para ser detectadas con SDSS con corrimientos al rojo 0,4. Esto crea un sesgo de medición.  
≈
 
Cuando tomo una muestra aleatoria de galaxias del mismo tamaño que el conjunto de datos QSO, obtengo un med_diff \ 0.018 que es ligeramente más alto que el conjunto completo, pero no lo suficiente como para explicar la brecha entre las dos poblaciones.  
≈
 
La siguiente figura muestra la función de distribución normalizada de las dos poblaciones.

Distribución de corrimiento al rojo
Podemos ver que la mayoría de las galaxias forman un pico alrededor de 0,10 mientras que los QSO se distribuyen razonablemente uniformemente hasta el corrimiento al rojo . Esto puede conducir a un sesgo de medición. En el caso de las galaxias, hemos entrenado nuestro árbol de decisión con desplazamientos al rojo objetivo de aproximadamente menos de 0,4. Como tal, las predicciones de este modelo no serán mayores que el valor objetivo máximo. Entonces, la diferencia máxima (o residual) para cada galaxia en este conjunto será mucho menor que el residual máximo para los QSO.  
≈
2.5
 
A menudo podemos obtener una visión más clara de esto al observar los desplazamientos al rojo pronosticados frente a los desplazamientos al rojo reales en un gráfico.

Pronosticado vs Real para QSOs y Galaxias

Hemos visto cómo los árboles de decisión son propensos a sobreajustar el modelo y cómo se puede usar la limitación de la profundidad máxima del árbol para evitar esto. Al comparar la precisión del modelo en el conjunto de entrenamiento con la del conjunto de prueba para diferentes profundidades de árbol, encontramos que una profundidad máxima de árbol de 19 era adecuada para nuestro modelo.

Analizamos -fold y los diversos métodos que se pueden usar para implementarla. -fold mitiga el riesgo de que el conjunto de entrenamiento tenga una población única o específica del conjunto de datos; Por ejemplo, si todos los datos de entrenamiento contenían QSO y el conjunto de pruebas eran galaxias regulares. -folds también le permite obtener una predicción para todos los puntos en su conjunto de datos.  
k
  
k
  
k
 
Concluimos observando la subpoblación de QSO y cómo su medición de precisión fue significativamente peor que la de las otras galaxias. En una inspección más cercana, encontramos que se trataba de un sesgo de medición resultante de la diferencia en el rango de corrimientos al rojo en cada población.

Es de esperar que haya aprendido todas las herramientas necesarias para implementar un árbol de decisiones en un problema de regresión propio. En el próximo módulo, veremos cómo se pueden usar los árboles de decisión para la clasificación. Los usaremos para clasificar las galaxias como elípticas, espirales o de fusión.